### Quickly review outputs of current model to find good samples for an eval dataset

In [1]:
from anki_ai.domain.model import Deck
import pandas as pd

In [2]:
deck = Deck()

In [3]:
deck.read_txt("../data/new_deck.txt")

In [4]:
for note in deck[:100]:
    if "```" in note.back:
        print(f"Front: {note.front}\nBack:  {note.back}\nTags:  {note.tags}\n")

Front: Create soft link
Back:  ```bash<br>$ ln -s <file> <link><br>```
Tags:  ['linux']

Front: Zip destination option
Back:  ```bash<br>$ unzip <file> -d <path><br>```
Tags:  ['linux']

Front: Extract zip files
Back:  ```bash<br>$ unzip <file><br>```
Tags:  ['linux']

Front: List directory content
Back:  ```bash<br>$ ls <path><br>```
Tags:  ['linux']

Front: Print text to terminal
Back:  ```bash<br>$ echo <text><br>```
Tags:  ['linux']

Front: Create new file
Back:  ```bash<br>$ touch <file><br>```
Tags:  ['linux']

Front: Create new directory
Back:  ```bash<br>$ mkdir <path><br>```
Tags:  ['linux']

Front: Search text for patterns
Back:  ```bash<br>$ grep <command><br>```
Tags:  ['linux']

Front: Get command manual/help
Back:  ```bash<br>$ man <command><br>```
Tags:  ['linux']

Front: Print current working directory
Back:  ```bash<br>$ pwd<br>```
Tags:  ['linux']

Front: Change directory
Back:  ```bash<br>$ cd <path><br>```
Tags:  ['linux']

Front: Move/Rename file/dir
Back:  ```bash

In [5]:
def missing_br_tag(text):
    "Not precise by any means. Need improvement."
    if "```" in text: 
        if "<br>" in text:
            return False
        else:
            return True

In [6]:
orig_deck = Deck()
orig_deck.read_txt("../data/Selected Notes v7.txt")

for note in deck[:100]:
    if missing_br_tag(note.back):
        orig_note = orig_deck.get(note.guid)[0]
        print(f"Note GUID: {note.guid}\n")
        print(f"Front: {orig_note.front}\nBack:  {orig_note.back}\nTags:  {orig_note.tags}\n")
        print(f"Front: {note.front}\nBack:  {note.back}\nTags:  {note.tags}\n")
        print("###################\n")

Note GUID: rJ!PVSQ/>l

Front: Command to remove delimiters
Back:  `ds&lt;delimiter&gt;`
Tags:  ['nvim']

Front: Remove delimiters
Back:  ```ds <delimiter>```
Tags:  ['nvim']

###################

Note GUID: Ah&KcViJ!v

Front: Command to change delimiters
Back:  `cs&lt;old delimiter&gt;&lt;new delimiter&gt;`
Tags:  ['nvim']

Front: Change Anki delimiters
Back:  ```\
Tags:  ['nvim']

###################

Note GUID: "q|WTQ*r#Ai"

Front: Command to add delimiters around a text object
Back:  `ys&lt;modifier&gt;&lt;text object&gt;&lt;delimiter&gt;`
Tags:  ['nvim']

Front: Add delimiters around text object
Back:  ```ys<modifier><text object><delimiter>```
Tags:  ['nvim']

###################

Note GUID: Ilg(9O88De

Front: Command to remove a delimiter wrapping an entire line
Back:  `ds&lt;delimiter&gt;`
Tags:  ['nvim']

Front: Remove line delimiter
Back:  ```ds<delimiter>```
Tags:  ['nvim']

###################

Note GUID: IlJu@%3i1Q

Front: Text object for a sentence
Back:  `s`
Tags:  ['nvim

There are a couple of good failure examples here already:

In [7]:
guids = [
    "rJ!PVSQ/>l",
    "Ah&KcViJ!v", 
    "IlJu@%3i1Q",
]
reasons = [
    "Using triple backtick quotes without specifying the language and adding newlines",
    "Mentioning the command is an Anki command when, in fact, is a nvim command",
    "Missing command",
]

for guid, reason in zip(guids, reasons):
    print(f"{reason}\n")
    note = deck.get(guid)[0]
    print(f"Front: {note.front}\nBack:  {note.back}\nTags:  {note.tags}\n")
    print("###################\n")

Using triple backtick quotes without specifying the language and adding newlines

Front: Remove delimiters
Back:  ```ds <delimiter>```
Tags:  ['nvim']

###################

Mentioning the command is an Anki command when, in fact, is a nvim command

Front: Change Anki delimiters
Back:  ```\
Tags:  ['nvim']

###################

Missing command

Front: Text object for a sentence
Back:  ```\
Tags:  ['nvim']

###################



Let's create an LLM judge to be able to identify this type of errors for us.

In [11]:
SYSTEM_MSG = r"""
Your job is to evaluate Anki notes, and classify notes that are not formatted correctly.

Examples of good notes:

Example 1:

    Front: Create soft link
    Back:  ```bash<br>$ ln -s <file> <link><br>```
    Tags:  ['linux']

Example 2:

    Front: Zip destination option
    Back:  ```bash<br>$ unzip <file> -d <path><br>```
    Tags:  ['linux']

Example 3:

    Front: Extract zip files
    Back:  ```bash<br>$ unzip <file><br>```
    Tags:  ['linux']

Example 4:

    Front: List directory content
    Back:  ```bash<br>$ ls <path><br>```
    Tags:  ['linux']

Examples of bad notes: 

Example 1:

    Front: Return to previous directory
    Back:  ```bash $ cd -```
    Tags:  ['linux']

    Reasoning: Missing newlines (<br> tags) in code block

Example 2: 

    Front: Remove delimiters
    Back:  ```ds <delimiter>```
    Tags:  ['nvim']

    Reasoning: Using triple backtick quotes without specifying the language and adding newlines (<br> tag) in code block

Example 3: 

    Front: Change Anki delimiters
    Back:  ```\
    Tags:  ['nvim']
    
    Reasoning: Mentioning the command is an Anki command when, in fact, it's a nvim command

Example 4: 

    Front: Text object for a sentence
    Back:  ```\
    Tags:  ['nvim']
    
    Reasoning: Missing command and not closing code block
"""

In [42]:
from anki_ai.domain.model import Note
from anki_ai.service_layer.services import get_chat_completion, ChatCompletionService, strip_tags
from typing import cast




def review_note(note: Note, chat: ChatCompletionService) -> Note:
    user_msg = f"""Front: {note.front}\nBack: {note.back}\nTags: {note.tags}"""

    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": user_msg},
    ]

    chat_response = chat.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        messages=messages,  # type: ignore
        temperature=0,
    )
    result: str = cast(str, chat_response.choices[0].message.content)

    print(user_msg)
    print(f"Eval: {result}\n")
    print("#######################\n")

In [43]:
chat = get_chat_completion()
for note in deck[:10]:
    review_note(note, chat)

Front: Headboard
Back: Headboard
Tags: ['english']
Eval: This note is not formatted correctly.

Reasoning: The front and back of the note are the same, which is not a good practice in Anki. The front should be a question or a prompt, and the back should be the answer or the information to be remembered. 

Additionally, the back of the note is not formatted as a code block, which is typically used to display commands or code snippets. It's also missing the newlines (<br> tags) that are commonly used to separate lines of code in a code block. 

Lastly, the note is missing a clear and concise description of the term "Headboard". It would be more effective to rephrase the front and back of the note to make it more engaging and easier to remember. 

Here's an example of how the note could be reformatted:

    Front: What is a headboard?
    Back:  ```bash<br>A headboard is a piece of furniture that is placed at the foot of a bed.<br>```
    Tags:  ['english']

#######################
Front: